<a href="https://colab.research.google.com/github/tomdaler/Machine-Learning/blob/main/Optimization/Bayesian_Optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Wine Quality, using RandomForestRegressor, RandomForestClassificator RandomizedSearchCV

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('winequality-red.csv')

df['quality']= df['quality']-3
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()

scaled_features = df.copy()

col_names = ['fixed acidity','free sulfur dioxide','total sulfur dioxide','alcohol','pH','residual sugar']
features = scaled_features[col_names]
df_new = scale.fit_transform(features)

scaled_features[col_names] = df_new


In [ ]:
X = scaled_features.drop(['quality'],axis=1)
y = scaled_features['quality']

X=X.drop(['alcohol', 'sulphates','pH','density','total sulfur dioxide'], axis=1)


In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=2)


In [ ]:
!pip install h2o
!pip install bayesian-optimization

     |████████████████████████████████| 176.2 MB 1.7 kB/s 
  Created wheel for h2o: filename=h2o-3.36.0.4-py2.py3-none-any.whl size=176245474 sha256=65671809f518c67f47f1e81e79fd82713ead126ad07ec6a46c0f3c7f9aec89cb
  Stored in directory: /root/.cache/pip/wheels/66/7c/3b/6558f625a9415a96f7f650d33f1def07b6e30b0c871e8407a1
Successfully built h2o
  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=f67e8be8610d849acaa2fe0a59b1e91f64993dbe19766d7febafa45cc57a9b67
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization


In [ ]:
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from bayes_opt import BayesianOptimization
h2o.init()
h2o.remove_all()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.14" 2022-01-18; OpenJDK Runtime Environment (build 11.0.14+9-Ubuntu-0ubuntu2.18.04); OpenJDK 64-Bit Server VM (build 11.0.14+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpuyxryze1
  JVM stdout: /tmp/tmpuyxryze1/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpuyxryze1/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.4
H2O_cluster_version_age:,8 days
H2O_cluster_name:,H2O_from_python_unknownUser_0h9evu
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [ ]:
data = h2o.upload_file('winequality-red.csv')
train_cols = [x for x in data.col_names if x not in ['quality']]
target = "quality"
train, test = data.split_frame(ratios=[0.7])


#train, test = df.split_frame(ratios=[0.7])

def train_model(max_depth, 
                ntrees,
                min_rows, 
                learn_rate, 
                sample_rate, 
                col_sample_rate):
    params = {
        'max_depth': int(max_depth),
        'ntrees': int(ntrees),
        'min_rows': int(min_rows),
        'learn_rate':learn_rate,
        'sample_rate':sample_rate,
        'col_sample_rate':col_sample_rate
    }
    model = H2OGradientBoostingEstimator(nfolds=5,**params)
    model.train(x=train_cols, y=target, training_frame=train)
    return -model.rmse()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
bounds = {
    'max_depth':(5,10),
    'ntrees': (100,200),
    'min_rows':(10,30),
    'learn_rate':(0.001, 0.01),
    'sample_rate':(0.5,0.8),
    'col_sample_rate':(0.5,0.8)
}

In [ ]:
optimizer = BayesianOptimization(
    f=train_model,
    pbounds=bounds,
    random_state=1,
)
optimizer.maximize(init_points=10, n_iter=50)

|   iter    |  target   | col_sa... | learn_... | max_depth | min_rows  |  ntrees   | sample... |
-------------------------------------------------------------------------------------------------
gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
|  1        | -0.6411   |  0.6251   |  0.007483 |  5.001    |  16.05    |  114.7    |  0.5277   |
gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
|  2        | -0.6694   |  0.5559   |  0.00411  |  6.984    |  20.78    |  141.9    |  0.7056   |
gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
|  3        | -0.6057   |  0.5613   |  0.008903 |  5.137    |  23.41    |  141.7    |  0.6676   |
gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
|  4        | -0.7195   |  0.5421   |  0.002783 |  9.004    |  29.37    |  131.3    |  0.7077   |
gbm Model Build progress: |█████

In [ ]:
optimizer.max

{'params': {'col_sample_rate': 0.7671344314874786,
  'learn_rate': 0.009785959643160268,
  'max_depth': 7.251777220993257,
  'min_rows': 17.765367664771894,
  'ntrees': 194.94105772340743,
  'sample_rate': 0.6746206311896739},
 'target': -0.5212552951937567}

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC
from bayes_opt import BayesianOptimization, UtilityFunction

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=2)


def black_box_function(C):

    # C: SVC hyper parameter to optimize for.
    model = SVC(C = C)
    model.fit(train_X, train_y)
    y_score = model.decision_function(test_X)
    f = roc_auc_score(test_y, y_score, multi_class='ovr')

    return f# Set range of C to optimize for.


In [ ]:
model = SVC(C = 5)
model.fit(train_X, train_y)
y_score = model.decision_function(test_X)
f = roc_auc_score(test_y, y_score, multi_class='ovr')  # ovr ovo raise

#clf = LogisticRegression(solver="liblinear").fit(X, y)
#>>> roc_auc_score(y, clf.predict_proba(X), multi_class='ovr')


ValueError: ignored

In [ ]:
# bayes_opt requires this to be a dictionary.
pbounds = {"C": [0.1, 10]}

# Create a BayesianOptimization optimizer,
# and optimize the given black_box_function.

optimizer = BayesianOptimization(f = black_box_function,
                                 pbounds = pbounds, verbose = 2,
                                 random_state = 4)

optimizer.maximize(init_points = 5, n_iter = 10)

print("Best result: {}; f(x) = {}.".format(optimizer.max["params"], optimizer.max["target"]))


|   iter    |  target   |     C     |
-------------------------------------


ValueError: ignored

In [ ]:
model1 = SVC(C = 8.505)
model1.fit(X_train_scaled, y_train)
y_score = model1.decision_function(X_test_scaled)
f = roc_auc_score(y_test, y_score)

print(f)
